In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, save_img
import numpy as np

In [5]:
def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)

    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    if p_spatial > 0.75:
        image = tf.image.transpose(image)

    # Rotates
    if p_rotate > 0.75:
        image = tf.image.rot90(image, k=3)  # rotate 270º
    elif p_rotate > 0.5:
        image = tf.image.rot90(image, k=2)  # rotate 180º
    elif p_rotate > 0.25:
        image = tf.image.rot90(image, k=1)  # rotate 90º

    # Pixel-level transforms
    if p_pixel_1 >= 0.4:
        # Manual contrast adjustment
        contrast_factor = 0.8
        image = (image - 0.5) * contrast_factor + 0.5
    if p_pixel_2 >= 0.4:
        # Manual brightness adjustment
        brightness_factor = 0.8
        image = image * brightness_factor
    if p_pixel_3 >= 0.4:
        # Manual saturation adjustment
        saturation_factor = 0.8
        image = (image - 0.5) * saturation_factor + 0.5

    return image

In [6]:
# Initialize your data generator with the `data_augment` function
datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=data_augment
)

In [7]:

# Define the directories
train_path = 'brain-tumor_dataset'
save_path = 'brain-tumor_dataset_augmented'

In [8]:

# Categories or classes
categories = ['clase_1', 'clase_2', 'clase_3']

# Create the save_path if it doesn't exist
os.makedirs(save_path, exist_ok=True)


In [9]:
#crear más imagenes
num_augmented_images = 5  # The number of augmented images to generate per original image

for category in categories:
    category_save_path = os.path.join(save_path, category)
    os.makedirs(category_save_path, exist_ok=True)
    
    image_files = os.listdir(os.path.join(train_path, category))
    
    for image_file in image_files:
        img_path = os.path.join(train_path, category, image_file)
        
        # Load the image
        image = load_img(img_path)
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)  # Add the batch dimension
        
        # Generate augmented images
        aug_iter = datagen.flow(image, batch_size=1, save_to_dir=category_save_path, save_prefix='aug', save_format='jpeg')
        
        # Save the specified number of augmented images
        for i in range(num_augmented_images):
            aug_iter.next()
print("Augmentation complete. Images saved to:", save_path)

Augmentation complete. Images saved to: brain-tumor_dataset_augmented
